Arbol elemental con libreria  rpart

Debe tener instaladas las librerias  data.table  ,  rpart   y rpart.plot

In [1]:
#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot



Aqui se debe poner la carpeta de SU computadora local

In [2]:
setwd("/home/nnm/Documents/Facu/1-Maestria/DMEF")  #Establezco el Working Directory

"/home/nnm/Documents/Facu/1-Maestria/DMEF"cargo el dataset

In [3]:
dataset  <- fread("./datasets/competencia1_2022.csv")

defino donde entreno y donde aplico el modelo

In [4]:
dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar
dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [14]:
dtrain[clase_ternaria == "BAJA+1"]

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
31127916,202101,1,0,0,42,277,2649.44,30930.84,2519.56,⋯,11,3485.96,-3448.72,0,5103,3002.88,2,0,598.23,BAJA+1
31328871,202101,1,0,0,83,293,335.32,17274.70,1699.39,⋯,25,48368.31,-44473.12,0,7490,2768.26,4,0,3964.74,BAJA+1
31406150,202101,1,0,0,69,296,567.94,14262.98,1246.04,⋯,4,0.00,-13090.55,0,4772,0.00,0,0,0.00,BAJA+1
31518298,202101,1,0,0,56,13,-149.22,893.80,303.60,⋯,4,0.00,-4926.60,0,377,3941.29,4,0,586.50,BAJA+1
31551292,202101,1,0,0,50,93,3002.50,6436.35,2564.83,⋯,4,0.00,-38246.48,0,2019,15901.17,6,0,9864.93,BAJA+1


genero el modelo,  aqui se construye el arbol

In [19]:
modelo  <- rpart(formula=    "clase_ternaria ~ .", #quiero predecir clase_ternaria a partir de el resto de las variables
                 data =      dtrain,  #los datos donde voy a entrenar
                 xval=       0,
                 cp=        -0.3, #esto significa no limitar la complejidad de los splits
                 minsplit= 50,   #minima cantidad de registros para que se haga el split
                 minbucket=  1,   #tamaño minimo de una hoja
                 maxdepth=  10 )  #PRUEBO  una profundidad de 10

Ahora grafico el arbol, va a ser muy frondoso porque la profundidad es de 10

In [20]:
#prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0)

Ahora aplico al modelo  a los datos muevos  y genero la salida para kaggle

aplico el modelo a los datos nuevos

In [21]:
prediccion  <- predict( modelo, dapply , type = "prob") #aplico el modelo

prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
cada columna es el vector de probabilidades 

agrego a dapply una columna nueva que es la probabilidad de BAJA+2

In [22]:
dapply[ , prob_baja2 := prediccion[, "BAJA+2"] ]

solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/40

In [23]:
dapply[ , Predicted  := as.numeric(prob_baja2 > 1/40) ]

Finalmente genero el archivo para Kaggle

creo la carpeta donde va el experimento

In [24]:
#dir.create( "./exp/KA2002" ) 

grabo el dataset

In [25]:
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/KA2002/KA2002_003.csv", 
        sep= "," )